In [6]:
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sneakers_ml.features.features import get_train_val_test
from sneakers_ml.models.onnx import save_sklearn_onnx

In [7]:
x_train, x_val, x_test, y_train, y_val, y_test = get_train_val_test("data/features/brands-classification-splits", "hog")

In [8]:
param_grid = {"C": [0.1, 1, 10], "gamma": ["scale", "auto"], "kernel": ["linear"]}
svc = SVC()
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1, n_jobs=-1)
grid_search.fit(x_train, y_train)
pred = grid_search.best_estimator_.predict(x_test)
print(f"Acc: {accuracy_score(y_test, pred)}")
print(f"F1-weighted: {f1_score(y_test,pred,average='weighted')}")
print(f"F1-macro: {f1_score(y_test,pred,average='macro')}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Acc: 0.7362534948741846
F1-weighted: 0.733982333125157
F1-macro: 0.7091623140216827


In [9]:
save_sklearn_onnx(grid_search.best_estimator_, x_train, "data/models/brands-classification/hog-svc.onnx")

In [10]:
param_grid = {"loss": ["log_loss", "hinge"], "alpha": [0.0001, 0.001]}
sgd = SGDClassifier()
grid_search = GridSearchCV(estimator=sgd, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1, n_jobs=-1)
grid_search.fit(x_train, y_train)
pred = grid_search.best_estimator_.predict(x_test)
print(f"Acc: {accuracy_score(y_test, pred)}")
print(f"F1-weighted: {f1_score(y_test,pred,average='weighted')}")
print(f"F1-macro: {f1_score(y_test,pred,average='macro')}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Acc: 0.7054986020503262
F1-weighted: 0.7041074267739038
F1-macro: 0.6736695136112966


In [11]:
grid_search.best_params_

{'alpha': 0.001, 'loss': 'log_loss'}

In [12]:
save_sklearn_onnx(grid_search.best_estimator_, x_train, "data/models/brands-classification/hog-sgd.onnx")

In [13]:
model = CatBoostClassifier(verbose=True, iterations=500, task_type="GPU")
model.fit(x_train, y_train, eval_set=(x_val, y_val))
pred = model.predict(x_test)

Learning rate set to 0.160002
0:	learn: 2.2855685	test: 2.3175174	best: 2.3175174 (0)	total: 423ms	remaining: 3m 30s
1:	learn: 2.1381811	test: 2.1931291	best: 2.1931291 (1)	total: 709ms	remaining: 2m 56s
2:	learn: 2.0337204	test: 2.1052822	best: 2.1052822 (2)	total: 998ms	remaining: 2m 45s
3:	learn: 1.9583682	test: 2.0414644	best: 2.0414644 (3)	total: 1.28s	remaining: 2m 38s
4:	learn: 1.8814049	test: 1.9755984	best: 1.9755984 (4)	total: 1.55s	remaining: 2m 33s
5:	learn: 1.8245495	test: 1.9298937	best: 1.9298937 (5)	total: 1.81s	remaining: 2m 29s
6:	learn: 1.7765564	test: 1.8909509	best: 1.8909509 (6)	total: 2.09s	remaining: 2m 27s
7:	learn: 1.7241868	test: 1.8533435	best: 1.8533435 (7)	total: 2.37s	remaining: 2m 25s
8:	learn: 1.6848430	test: 1.8234286	best: 1.8234286 (8)	total: 2.64s	remaining: 2m 24s
9:	learn: 1.6501734	test: 1.7975510	best: 1.7975510 (9)	total: 2.91s	remaining: 2m 22s
10:	learn: 1.6179001	test: 1.7747034	best: 1.7747034 (10)	total: 3.18s	remaining: 2m 21s
11:	learn: 

In [14]:
print(f"Acc: {accuracy_score(y_test, pred)}")
print(f"F1-weighted: {f1_score(y_test,pred,average='weighted')}")
print(f"F1-macro: {f1_score(y_test,pred,average='macro')}")

Acc: 0.6952469711090401
F1-weighted: 0.6787502538422535
F1-macro: 0.6270385458127209


In [15]:
model.save_model(
    "data/models/brands-classification/hog-catboost.onnx",
    format="onnx",
    export_parameters={
        "onnx_domain": "ai.catboost",
        "onnx_model_version": 1,
        "onnx_doc_string": "iterations=500 default model",
        "onnx_graph_name": "CatBoostModel_for_MultiClassification",
    },
)